## Capstone Project Assignment 2 :Segmenting and Clustering Neighborhoods in Toronto 3


In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

In [2]:
link = "https://raw.githubusercontent.com/Laincoubert8/Coursera_Capstone/master/Week_3/Assignment2.csv"
neighborhoods = pd.read_csv(link,index_col=0)

address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_x = location.longitude

map_toronto = folium.Map(location=[latitude_x, longitude_x], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude_x'], neighborhoods['Longitude_x'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF7F7F',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [3]:
LIMIT = 100
CLIENT_ID = 'OEM1KGPBAE1DJPP2NRJK2RIYX5IUBWEVNV1WG441OXHJCNEA'
CLIENT_SECRET = 'PIJRVPMHDQPCVHW0GR0VU3BDX4BUGRNQQGSLKFKQBE1D2KOX' 
VERSION = '20200801' 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude_x', 
                  'Neighborhood Longitude_x', 
                  'Venue', 
                  'Venue Latitude_x', 
                  'Venue Longitude_x', 
                  'Venue Category']
    
    return(nearby_venues)

streets_toronto = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude_x'],
                                   longitudes=neighborhoods['Longitude_x'],
                                  )

Rouge ,Malvern
Highland Creek ,Rouge Hill ,Port Union
Guildwood ,Morningside ,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park ,Ionview ,Kennedy Park
Clairlea ,Golden Mile ,Oakridge
Cliffcrest ,Cliffside ,Scarborough Village West
Birch Cliff ,Cliffside West
Dorset Park ,Scarborough Town Centre ,Wexford Heights
Maryvale ,Wexford
Agincourt
Clarks Corners ,Sullivan ,Tam O'Shanter
Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview ,Henry Farm ,Oriole
Bayview Village
Silver Hills ,York Mills
Newtonbrook ,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park ,Don Mills South
Bathurst Manor ,Downsview North ,Wilson Heights
Northwood Park ,York University
CFB Toronto ,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens ,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West ,

In [8]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

city_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude_x'],
                                   longitudes=neighborhoods['Longitude_x'],
                                  )





Rouge ,Malvern
Highland Creek ,Rouge Hill ,Port Union
Guildwood ,Morningside ,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park ,Ionview ,Kennedy Park
Clairlea ,Golden Mile ,Oakridge
Cliffcrest ,Cliffside ,Scarborough Village West
Birch Cliff ,Cliffside West
Dorset Park ,Scarborough Town Centre ,Wexford Heights
Maryvale ,Wexford
Agincourt
Clarks Corners ,Sullivan ,Tam O'Shanter
Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview ,Henry Farm ,Oriole
Bayview Village
Silver Hills ,York Mills
Newtonbrook ,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park ,Don Mills South
Bathurst Manor ,Downsview North ,Wilson Heights
Northwood Park ,York University
CFB Toronto ,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens ,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West ,

In [9]:
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

city_onehot['Neighborhood'] = city_venues['Neighborhood'] 

fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]


In [10]:
city_grouped = city_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 10


indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = city_grouped['Neighborhood']

for ind in np.arange(city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide ,King ,Richmond",Coffee Shop,Café,Clothing Store,Hotel,Restaurant,Gym,Bar,Thai Restaurant,Steakhouse,Salad Place
1,Agincourt,Breakfast Spot,Lounge,Latin American Restaurant,Clothing Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,"Agincourt North ,L'Amoreaux East ,Milliken ,St...",Gym,Playground,Park,Greek Restaurant,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
3,"Albion Gardens ,Beaumond Heights ,Humbergate ,...",Grocery Store,Pizza Place,Liquor Store,Sandwich Place,Fast Food Restaurant,Fried Chicken Joint,Pharmacy,Beer Store,Drugstore,Donut Shop
4,"Alderwood ,Long Branch",Pizza Place,Coffee Shop,Gym,Pub,Sandwich Place,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner


In [11]:
kclusters = 3

city_grouped_clustering = city_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = neighborhoods

city_merged = city_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')


In [12]:
map_clusters = folium.Map(location=[latitude_x, longitude_x], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(city_merged['Latitude_x'], city_merged['Longitude_x'], city_merged['Neighbourhood'], city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF7F7F',
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

In [13]:
city_merged.loc[city_merged['Cluster Labels'] == 0, city_merged.columns[[1] + list(range(3, city_merged.shape[1]))]]

,Borough,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,43.806686,-79.194353,43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,Scarborough,43.784535,-79.160497,43.784535,-79.160497,0.0,Moving Target,Bar,Home Service,Women's Store,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,Scarborough,43.763573,-79.188711,43.763573,-79.188711,0.0,Rental Car Location,Restaurant,Mexican Restaurant,Breakfast Spot,Intersection,Bank,Medical Center,Electronics Store,Diner,Dim Sum Restaurant
3,Scarborough,43.770992,-79.216917,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Mexican Restaurant,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,Scarborough,43.773136,-79.239476,43.773136,-79.239476,0.0,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Thai Restaurant,Fried Chicken Joint,Doner Restaurant,Dog Run
5,Scarborough,43.744734,-79.239476,43.744734,-79.239476,0.0,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
6,Scarborough,43.727929,-79.262029,43.727929,-79.262029,0.0,Discount Store,Coffee Shop,Hobby Shop,Department Store,Convenience Store,Chinese Restaurant,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner
7,Scarborough,43.711112,-79.284577,43.711112,-79.284577,0.0,Bus Line,Bakery,Park,Bus Station,Soccer Field,Intersection,Metro Station,Ice Cream Shop,Discount Store,Diner
8,Scarborough,43.716316,-79.239476,43.716316,-79.239476,0.0,Motel,American Restaurant,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
9,Scarborough,43.692657,-79.264848,43.692657,-79.264848,0.0,College Stadium,Skating Rink,Café,General Entertainment,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


In [14]:
city_merged.loc[city_merged['Cluster Labels'] == 1, city_merged.columns[[1] + list(range(3, city_merged.shape[1]))]]

,Borough,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,43.815252,-79.284577,43.815252,-79.284577,1.0,Gym,Playground,Park,Greek Restaurant,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
20,North York,43.757490,-79.374714,43.757490,-79.374714,1.0,Park,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
21,North York,43.789053,-79.408493,43.789053,-79.408493,1.0,Park,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
23,North York,43.752758,-79.400049,43.752758,-79.400049,1.0,Convenience Store,Park,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
30,North York,43.737473,-79.464763,43.737473,-79.464763,1.0,Park,Airport,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
40,East York,43.685347,-79.338106,43.685347,-79.338106,1.0,Park,Convenience Store,Intersection,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Department Store
44,Central Toronto,43.728020,-79.388790,43.728020,-79.388790,1.0,Park,Bus Line,Swim School,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store,Dog Run
50,Downtown Toronto,43.679563,-79.377529,43.679563,-79.377529,1.0,Park,Playground,Trail,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Electronics Store,Donut Shop,Doner Restaurant,Dance Studio
74,York,43.689026,-79.453512,43.689026,-79.453512,1.0,Park,Pool,Women's Store,Greek Restaurant,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
90,Etobicoke,43.653654,-79.506944,43.653654,-79.506944,1.0,Park,River,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Event Space


In [15]:
city_merged.loc[city_merged['Cluster Labels'] == 2, city_merged.columns[[1] + list(range(3, city_merged.shape[1]))]]

,Borough,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,43.711695,-79.416936,43.711695,-79.416936,2.0,Garden,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Women's Store,Deli / Bodega


### We have created 3 clusters where the first one contains most of the neighbours and the last one only one.